**Assumptions**
- The knowledge base is already prepared for the agent.  (not all data needs to be dynamcially fetched on run-time)
- A invocation run does not require context from a previous run. No chat history -> No tweaks/edits
- We are only considering NSE for live data

**Cheat Sheet for Qualitative Analysis**

| Metric / Insight     | Signal To Look For                        |
| -------------------- | ----------------------------------------- |
| Total Revenue        | Growth trend, seasonal strength/weakness  |
| Operating Margin     | Cost control or inflationary pressure     |
| Net Profit / EPS     | Health of the business, investor returns  |
| Recurring Themes     | Strategic focus areas or persistent risks |
| Management Sentiment | Confidence vs. hedging                    |
| Forward Guidance     | Directional clues (explicit or subtle)    |
| Segment Performance  | What’s winning or underperforming         |


## Observations:
    1. YFinance tools + Simple ReAct Agent -> [2/5] (High token usage, incomplete data points)

In [60]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os

load_dotenv()


llm = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai",
    temperature=0.2,
    api_key=os.getenv("GOOGLE_API_KEY"),
)

In [ ]:
# from langchain_mcp_adapters.client import MultiServerMCPClient

# client = MultiServerMCPClient(
#     {
#         "finance": {
#             "url": "https://yfinance-mcp-owvg.onrender.com/mcp/",
#             "transport": "streamable_http",
#         }
#     }
# )

# tools = await client.get_tools()


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

collection_name = "transcripts"

client = QdrantClient(url="http://localhost:6333")
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)


def search_docs(topics_list: list[str]):
    """Search for a list of metrics/values/figures or general information in the company X's earnings call data."""
    results = []
    for query in topics_list:
        result = vector_store.similarity_search(query, k=4)
        results.append(
            {
                "query": query,
                "top_results": [results.page_content for results in result],
            }
        )
    return results


agent = create_react_agent(
    model=llm,
    tools=[search_docs],
    debug=True,
    prompt="You are an expert financial analyst. Use the provided tools to research for a task, and ALWAYS take the step-by-step approach. Reflect after each step to decide whether you have everything you need.",
)

In [6]:
await agent.ainvoke(
    {
        "messages": {
            "role": "user",
            "content": "Analyze the financial reports for ticker = 'TCS.NS' for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.",
        }
    }
)

[values] {'messages': [HumanMessage(content="Analyze the financial reports for ticker = 'TCS.NS' for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.", additional_kwargs={}, response_metadata={}, id='01c334f9-2caa-44d9-87ac-4a4c65738cc5')]}
[updates] {'agent': {'messages': [AIMessage(content="Okay, I can analyze the financial reports for 'TCS.NS' and provide a qualitative forecast. Here's my plan:\n\n1.  **Identify Key Financial Trends:** I'll use the `search_docs` tool to look for information on revenue growth, margins, and other relevant financial metrics.\n2.  **Summarize Management's Outlook:** I'll use the `search_docs` tool to find statements from management regarding their outlook for the next quarter.\n3.  **Highlight Risks and Opportunities:** I'll use the `

{'messages': [HumanMessage(content="Analyze the financial reports for ticker = 'TCS.NS' for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.", additional_kwargs={}, response_metadata={}, id='01c334f9-2caa-44d9-87ac-4a4c65738cc5'),
  AIMessage(content="Okay, I can analyze the financial reports for 'TCS.NS' and provide a qualitative forecast. Here's my plan:\n\n1.  **Identify Key Financial Trends:** I'll use the `search_docs` tool to look for information on revenue growth, margins, and other relevant financial metrics.\n2.  **Summarize Management's Outlook:** I'll use the `search_docs` tool to find statements from management regarding their outlook for the next quarter.\n3.  **Highlight Risks and Opportunities:** I'll use the `search_docs` tool to identify any risks or

In [ ]:
# agent.invoke(
#     {
#         "messages": {
#             "role": "user",
#             "content": "Analyze the financial reports and transcripts for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.",
#         }
#     }
# )

In [15]:
file_text = """Ind-AS & IFRS ₹ 
Strong TCV in a seasonally challenging Q3 positions TCS for Long-Term growth - - - - - 
Revenue at ₹63,973 crore | Growth of 5.6% YoY, +4.5% in CC 
Growth led by Consumer Business Group (+1.1%), Energy, Resources and Utilities 
(+3.4%) and Regional Markets (+40.9%) 
Growth Markets continue to lead: India (+70.2%), Middle East & Africa (+15.0%), 
Latin America (+7.0%), Asia Pacific (+5.8%)  
Operating Margin at 24.5%; Net Margin at 19.4% 
Strong TCV at US$10.2 Billion, Book to Bill ratio of 1.4 
MUMBAI, January 9, 2025: Tata Consultancy Services (BSE: 532540, NSE: TCS) reported its 
consolidated financial results according to Ind AS and IFRS, for the quarter ending December 31, 2024. 
Highlights of the Quarter Ended December 31, 2024 
• Revenue at ₹63,973 crore, +5.6% YoY, +4.5% YoY in Constant Currency 
• Operating Margin at 24.5%; a decline of 50 bps YoY*, sequential improvement of 40 bps 
• Net Income at ₹12,380 crore, +5.5% YoY* | Net Margin at 19.4% 
• Net Cash from Operations at ₹13,032 crore i.e. 105.3% of Net Income 
• Workforce strength: 607,354 | LTM IT Services attrition rate at 13.0% 
• Diverse and inclusive workplace: Women in the workforce: 35.3% |152 Nationalities 
• Dividend per share: ₹ 76.00 including ₹ 66.00 as special dividend | Record date 17/01/2025 | 
Payment date 03/02/2025 
*excludes settlement of legal claim 
K Krithivasan, Chief Executive Officer and Managing Director, said: “We are pleased with the excellent 
TCV performance in Q3 which was well-rounded across industries, geographies and service lines 
lending good visibility to long-term growth. BFSI and CBG returning to growth, continued stellar run of 
Regional Markets and early signs of revival in discretionary spend in some verticals give us confidence 
for the future. Our continuing investments in upskilling, AI/Gen AI Innovations and partnerships sets us 
up to capture the promising opportunities ahead.” 
Samir Seksaria, Chief Financial Officer, said: “In a quarter that saw significant cross-currency volatility, 
TCS’s strong execution, cost management and deft currency risk management helped deliver healthy 
margin improvement and free cash flows. Disciplined investments in talent and infrastructure should 
lend good support to long-term business growth.” 
Page 1 of 14 
Page 2 of 14 
 
 
Milind Lakkad, Chief HR Officer, said: “We promoted over 25,000 associates this quarter which 
brought the total promotions this financial year to more than 110,000. We continue to invest in 
employee upskilling and overall well-being. Our campus hiring for the year is going according to plan 
and preparations are afoot to onboard a higher number of campus hires next year”. 
 
Q3 FY25 Segment Highlights 
Growth by Domain 
Industry 
Composition (%) Y-o-Y 
CC Growth 
(%) Q3 FY24 Q3 FY25 
 BFSI  31.7 30.5  0.9 
 Consumer Business  15.7 15.3  1.1 
 Life Sciences & Healthcare  10.9 10.1 - 4.3 
 Manufacturing  8.7 8.4  0.4 
 Technology & Services  8.4 8.0 - 0.4 
 Communication & Media  6.7 5.8 - 10.6 
 Energy, Resources and Utilities  5.7 5.6  3.4 
 Regional Markets & Others  12.2 16.4  40.9 
 Total  100.0 100.0  4.5 
 
Growth by Markets 
Geography 
Composition (%) Y-o-Y 
CC 
Growth 
(%) Q3 FY24 Q3 FY25 
 Americas        
 North America  50.6 47.7 - 2.3 
 Latin America  2.1 1.9  7.0 
 Europe        
 UK  16.4 16.6  4.1 
 Continental Europe  15.0 13.9 - 1.5 
 Asia Pacific  7.8 7.8  5.8 
 India  6.1 9.8  70.2 
 MEA  2.0 2.3  15.0 
 Total  100.0 100.0 4.5 
Services:  
Clients prioritized initiatives that accelerated technology modernization, factory of the future, 
business transformation, technology simplification and sustainability. 
Cyber Security, AI.Cloud and TCS Interactive led the growth this quarter. 
Win themes across key deals involved manufacturing IT operations, managed services, Operating 
model transformation, vendor consolidation, CX transformation, legacy modernization, and Gen AI 
led transformation.  
With increased regulatory scrutiny and legislative action, responsible AI adoption is top of mind for 
our clients. We have launched an innovative offering, TCS 5A Framework for Responsible AI with our 
partner AWS. This is the first of its kind in the industry. It helps identify and mitigate AI risks 
holistically. 
• AI.Cloud: Gen AI/AI and Cloud services continued to see significant growth for us this 
quarter. Clients are investing in Agentic AI adoption, building robust data foundation and 
taking a value chain-based approach to AI and Gen AI led transformation. We actively 
engaged with clients on AI/Gen AI led software engineering, legacy modernization and 
AI.Ops.  We saw an increase in successful production deployment of AI/Gen AI engagements 
leading to greater business certainty and confidence for our clients. Cloud services continued 
to see good traction in legacy Modernization, Data Platform Modernization and Technology 
Landscape Simplification. 
• Cyber Security: Cyber Security services and solutions saw strong growth. Growth was led by 
Technology, Software and Services and BFSI industry units. Clients continued to focus on 
Identity and Access Management, Governance, Risk & Compliance (GRC), Managed 
Detection & Response (MDR) and Cloud Security. We are also seeing good traction in Secure 
Gen AI and vulnerability management. 
• TCS Interactive: TCS interactive services saw good traction this quarter. BFSI and CBG led 
growth in industry segments.  Businesses and marketers are investing in personalization, 
data harmonization, marketing operations, and orchestrate the appropriate technology and 
data foundations. To capitalize on these opportunities, we continue to invest in creative 
design, content services, leading technology partnerships, Gen AI solutions, and niche talent 
development to remain future-ready and relevant.  
• IoT & Digital Engineering: IoT & Digital Engineering continues to see steady investments in 
Factory of the Future (Smart Manufacturing including Operational technology (OT) 
modernization), IoT Enabled Connected Services & Assets and Digital Thread. This quarter 
saw a good number of large deals for our services. Key services that saw traction included 
intelligent product engineering design, support and services, Manufacturing IT systems and 
Industry 4.0. Demand is growing for integrating AI & Gen AI with established services.  
• Enterprise Solutions: Clients continue to invest in their digital core ERP modernization. They 
are partnering with TCS in their business and CX transformation journey for solutions in 
supply chain optimization, claims modernization, sustainability and reimagining sales & 
service channels. Gen AI is an area of interest across industries. Our transformation 
approach, contextual knowledge and industry pre-configured solutions including TCS 
Page 3 of 14 
Crystallus™ are enabling customers to accelerate their growth and transformation journeys. 
BFSI, TSS and CBG led growth in Industry segments. 
• Cognitive Business Operations: In this quarter, we saw multiple large wins across end-to
end IT IS services, Digital Workplace, Data Center, Network, Customer Experience, HR and 
Finance. Key themes fueling growth are operating model transformation, Vendor 
Consolidation, Customer Experience, Gen AI adoption, First time outsourcing and Integrated 
operations. Some of the deals won were proactive without competition. CBG and BFSI 
industry segments led growth. 
Key Highlights 
• Partnered with Landis+Gyr to integrate our award-winning enterprise-level energy and 
emission management system TCS Clever EnergyTM with their advanced smart metering and 
grid management technologies, creating a comprehensive energy management solution for 
utilities. The partnership will enable utilities to meet the complex energy demands of their 
commercial and industrial clients while offering them advanced tools to optimize energy usage 
and achieve sustainability targets. 
• Extended partnership with Telenor Denmark (TnDK), the second largest mobile operator in 
the Danish market, to provide ITIS managed services, handling TnDK’s IT infrastructure from 
its delivery centre in Europe over the next five years, to deliver significant operational cost 
reductions and improved IT infrastructure. 
• Selected by Bank of Bhutan to modernize the bank’s digital core for enhanced customer 
service. TCS will implement its TCS BaNCS™ Global Banking Platform to help the bank optimize 
operations and seamlessly integrate with the various public services within the digital 
ecosystem of the country. 
• Signed a multi-year deal to help Air France-KLM, to become the most data-centric airline group 
in the world. TCS will modernize the airline group's data by moving it to cloud and this shift 
will help Air France-KLM exit data centers and harness the strength of the cloud, supporting a 
sustainable and adaptive aviation industry. The new data architecture will enable the use of 
data to enhance operations, drive decisions, improve functionality, and gain efficiency. 
• Extended contract with SPARSH, to continue to streamline and enhance pension services for 
30 Lakh Plus defence pensioners in India. TCS has achieved significant reductions in pension 
processing times and implementing the One Rank One Pension (OROP) scheme, benefiting 1.8 
million pensioners. 
• Jabil, a leading engineering, manufacturing, and supply chain solutions company has selected 
TCS as a partner to further solidify its digital transformation agenda. TCS will support the 
simplification of manufacturing IT systems to better rationalize, streamline and secure Jabil’s 
manufacturing applications ecosystem and enterprise architecture for operational efficiencies. 
This project will allow Jabil to focus and invest in its core competencies. 
• Secured a 15-year contract with Ireland’s Department of Social Protection (DSP) to implement 
and support the country’s new Auto Enrolment Retirement Savings Scheme, known as the “My 
Future Fund.” This initiative will provide a comprehensive, end-to-end digital solution for 
automatic enrolment of nearly 800,000 workers in Ireland.   
Page 4 of 14 
• Selected by Canada’s largest worker compensation board, Workplace Safety and Insurance 
Board (WSIB), as a strategic partner to transform the enterprise quality assurance landscape. 
As part of the expanded partnership, TCS will leverage its expertise in Guidewire services, 
cloud, data, and AI to help WSIB realize its objective of delivering better, easier, and faster 
services to workers and businesses in Ontario. 
• Selected by Postnord Group AB, postal and logistics services company in Nordics as their 
strategic partner to transform their operating model. TCS team helped establish the roadmap 
for the transformation and will now embark on the first phase of the journey - centralizing 
operations in production. This will ensure best-in-class services and continuous innovation, 
deliver significant productivity gains and superior user experiences. 
• Entered a multi-year deal with a leading US life and annuities provider to transform their 
business operations with TCS’ AI-powered solutions and industry-proven operational 
efficiency levers. As part of the partnership, TCS will leverage its extensive domain knowledge, 
IP assets from its Cognitive Automation Platform, and a suite of AI/ML solutions, to help the 
f
 irm achieve straight through processing, improve service quality and customer experience. 
• Selected by global appliances manufacturer for end-to-end managed services for 
Applications, Infrastructure, and Cyber Security services globally. TCS will elevate the customer 
experience by deploying enterprise observability, spanning the entire business value chain and 
automate the process through the TCS ignio platform. 
• Chorley Building Society has partnered with TCS to transform its core banking and front-end 
digital platform, using TCS BaNCS™, a digital home lending solution for mortgage origination 
and a mobile app to enhance user experience, support Chorley’s growth plans and improve 
operational efficiency. 
• TCS has been chosen by a leading Malaysian telecommunications provider to transform their 
wireline Enterprise and Wholesale Business Support Systems (BSS) to support their strategic 
business growth objectives, using a cloud-native platform powered by TCS HOBS and TCS 
TwinX's AI-driven business simulation capabilities. This partnership aims to enhance customer 
experience, accelerate time-to-market for new products and services, optimize operations, 
and strengthen the provider's market share in Malaysia. 
• A US-based telecom major has entered a 5-year strategic relationship with TCS to provide 
managed services for specific solutions. We will leverage our investments in next gen AI 
technologies and platform-based operations, including pre-configured and integrated 
AI/GenAI solutions to drive Operating Model Transformation, enhance customer experience 
and business agility for business customers across industries. 
• Selected by a large UK-based investment platform provider to enhance and manage their 
digital platform for advisors and customers.  As part of the multi-year partnership, TCS will 
implement a next-generation engineering-led operating model designed to enhance and 
optimize digital platform capabilities to cater to the evolving business needs and elevate user 
experiences. By leveraging advanced AI solutions, TCS will ensure proactive monitoring of the 
system health and enable self-healing. The expanded partnership reflects TCS’ commitment to 
delivering cutting-edge solutions to help the firm drive operational excellence, innovation, and 
customer satisfaction. 
Page 5 of 14 
• A leading provider of hospitality services has entrusted TCS to manage their entire business 
applications portfolio across all operating countries. TCS will drive Machine First™ intelligent 
operations to bring hyper-automation, DevOps, observability, and cloud-native capabilities 
and will also manage the underlying infrastructure, data centers, cloud, cyber-security and the 
service desk. This will enhance operational efficiencies and deliver significant savings. 
• Selected by a leading American grocery retailer to transform their Demand Planning & 
Replenishment operations. TCS will provide business process services for “center of store” (not 
fresh or frozen) categories and drive digitalization, automation, analytics & AI to improve 
decision making and forecast accuracy, while lowering forecast bias. This will help increase 
distribution center to store fill rate, and operational efficiencies.  
• Selected as the exclusive strategic growth & transformation partner to help a leading Canadian 
retail corporation and subsidiary of a U.S. based multinational retail conglomerate on their 
growth and transformation journey. TCS will consolidate the work of 20+ vendors to deliver 
services across Supply Chain, Stores, Pharmacy, Distribution Centers & Corporate. TCS will also 
work on transforming the data architecture to future proof the business leveraging Agentic AI 
and Gen AI. This will deliver significant efficiencies, improve agility and scalability and ensure 
continuous improvement through innovation. 
• TCS extended the strategic partnership with a leading global industrial manufacturing and 
technology company to provide Enterprise Application Support services including SAP, 
Salesforce, Contact Center, Service Desk, Engineering & eCommerce Applications and 
Customer portals.  We will bring the automation-driven approach to optimize the run cost, 
improve system availability & observability, and enhance customer & user experience. 
• Selected by global leader in flavors, fragrances, food ingredients company to work on a one
of-its-kind initiative to determine the Protein Yield and molecular discovery for fragrances and 
f
 lavors using Nvidia’s BioNeMo framework. It is expected to cut down the research cycle time 
upto 5x by eliminating trial and error experiments. This framework leverages TCS' Pace Port™ 
and delivery ecosystem to target protein structure prediction, which is crucial for 
understanding protein function and designing new therapeutic molecules. 
• Selected by a leading European airline group, as their strategic innovation partner to deliver 
tailored AI solutions. TCS will onboard its WisdomNext™ platform to enable meaningful 
innovation across the organization and kick-start the journey by creating a Gen AI based chat 
interface which will assist associates in responding to emails, classifying and validating 
documents. This will set the stage for impactful, long-term AI-driven transformations across 
business units. 
Customer Speak  
“In a rapidly evolving industry, we are committed to becoming a champion of air transport in Europe, 
while fulfilling our role as a frontrunner of sustainable aviation. We are pleased to open a new and 
exciting chapter with our long-time partner, TCS, to make the most out of our data and new 
technologies to become even more data-centric and agile to meet and exceed our customers’ 
expectations.” 
Pierre-Olivier Bandet, EVP & Group CIO, Air France-KLM Group 
Page 6 of 14 
“We are happy to reinforce our partnership with TCS with the selection of TCS BaNCS™ Global Banking 
Platform. We remain committed to continuously innovating and enhancing customer experience. This 
translates into offering personalized, comprehensive, high-performing financial products and services, 
including the capabilities required to support the country-wide digitization and digital currency 
initiatives by the Kingdom of Bhutan. The selection of the TCS BaNCS™ Global Banking Platform that 
is designed and built on a modern technology stack with ecosystem enablers reflects this philosophy. 
The solution will also serve as a platform for future programs such as the implementation of AI 
technologies across operational workflows to boost productivity.” 
Dorji Kadin, Chief Executive Officer, Bank of Bhutan 
“Our partnership with TCS has been an important factor in driving innovation and ensuring resilience 
in our IT infrastructure. By continuing our collaboration, we are confident that TCS will continue to 
deliver the strategic insights and technological expertise required to navigate the complexities of our 
evolving business environment.” 
Louise Haurum, Chief Technology Officer, Telenor Denmark 
“This renewed agreement positions us to leverage TCS's cutting-edge technologies and innovations, 
further enhancing our operational capabilities and competitive edge. A long-term arrangement such 
as this ensures stability and allows us to strategically plan and allocate resources to continue bridging 
the financial gap for unbanked and underserved populations. This collaboration aligns seamlessly with 
the Government's vision of ‘Sabka Saath, Sabka Vikas, Sabka Vishwas, Sabka Prayaas’, driving us closer 
to building an Aatmanirbhar Bharat. Through innovation, dedication, and collective effort, we aim to 
bring inclusive financial services to every corner of the nation, fulfilling the aspirations of millions and 
contributing to India's self-reliance and growth.” 
Vimal Kumar Negi, General Manager, Financial Inclusion and Corporate Social Responsibility, BoB 
"As we continue our partnership with TCS in digitalizing the IT operating model at Advantage Solutions, 
TCS has delivered several significant milestones in 2024, leading to the enterprise-wide rollout of the 
new service management tool (ServiceNow), the launch of a cognitive IT help bot, and the stabilization 
of RunOps for IT service desk delivery. These accomplishments have greatly improved the customer 
experience, as evidenced by enhanced service desk metrics such as call service rate, average speed to 
answer, and first contact resolution rate. The TCS deliverables have also facilitated the rapid adoption 
of the new digital ticketing platform and improved the accuracy of provisioning devices and 
accessories to Advantage field reps. I’d like to extend a heartfelt thank you to for their invaluable 
support in helping us achieve these milestones.”  
Jey Perumalswamy, Vice President – Technology, Advantage Solutions Inc. 
Page 7 of 14 
Research and Innovation 
As on 31st December 2024, the company had applied for 8,549 patents, including 195 applied during 
the quarter and been granted 4,585 patents including 216 during the quarter. 
Human Resources 
TCS’ workforce stood at 607,354 as on December 31st. The employee base is very diverse, with 35.3% 
women and with 152 nationalities. 
TCSers have clocked 40.1 million learning hours and acquired 3.8 million competencies YTD. IT 
services’ attrition was at 13% for the last twelve months.  
Awards and Global Recognition 
• TCS awarded the ‘Best Employer Brand’ in the Indian corporate sector at the LinkedIn Talent 
Awards 2024. 
• TCS Ireland recognized as ‘Workplace of the Year’ at the Letterkenny Chamber Annual 
Business Awards. 
• TCS China awarded the ‘Best Employers in Greater China 2024’ by HRoot 2024. 
• TCS LATAM recognized by the International Organization of Human Capital Managers (DCH) 
for nurturing young talent and showing commitment to the wellbeing of associates. 
• TCS Malaysia awarded Life at Work Award 2024 by the Ministry of Human Resources & 
Talent Corp, highlighting TCS’ commitment to advancing Diversity, Equity and Inclusion. 
• TCS ranked #1 for Customer Satisfaction in France, by Whitelane Research. 
• Awarded Data and Analytics Partner of the Year by Amazon Web Services in the Global 
Consulting category. 
• Awarded the Intellectual Asset Management (IAM)’s Asia IP Elite Award 2024 for 
demonstrating a sustained commitment to best-in-class IP. 
• Recognized by the United States Chamber of Commerce Foundation as the Best Corporate 
Steward for 2024. 
• TCS China recognized in CSR China Top 100 under Corporate Social Responsibility by China 
Education List. 
• TCS Thailand awarded Corporate Social Impact Recognition by the American Chamber of 
Commerce (AMCHAM) Thailand. 
• Received 10 Brandon Hall Awards recognizing leadership in services, products, and internal 
management approaches, demonstrating ongoing innovation across the TCS business. 
Page 8 of 14 
Page 9 of 14 
 
 
Industry Analyst Assessments 
TCS was ranked a Leader in 62 competitive assessments published by leading research firms in Q3 
FY25. In 5 of these, marked with an asterisk in the table below, TCS was positioned the foremost 
leader or ranked #1. 
Analyst 
Firm 
Firm Wise 
Report Type Report Title 
IDC MarketScape 
IDC MarketScape: Worldwide Aftermarket Service Life-Cycle 
Management Systems Integrators/Business Process Outsourcing 
2024–2025 Vendor Assessment* 
IDC MarketScape IDC MarketScape: Worldwide Digital Workplace Services 2024 
Vendor Assessment* 
Frost & 
Sullivan Radar Outsourced Digital Workplace Servies 2024* 
NelsonHall NEAT Quality Engineering 2024* 
IDC MarketScape IDC MarketScape: Middle East and Africa Cloud Professional 
Services 2024 Vendor Assessment* 
Gartner Magic 
Quadrant Magic Quadrant for Custom Software Development Services 
HFS Horizons HFS Horizons: Sustainability Services, 2024 
Avasant RADAR Nordics Digital Services 2024–2025 RadarView™ 
Avasant RADAR Digital Talent Capability 2024 RadarView™ 
IDC MarketScape IDC MarketScape: Worldwide Service Providers for Utilities 
Customer Operations 2024 Vendor Assessment 
Everest PEAK Matrix Semiconductor Engineering Services PEAK Matrix® Assessment 2024 
Avasant RADAR CPG Digital Services 2024 RadarView™ 
IDC MarketScape IDC MarketScape: Worldwide Aftermarket Service Life-Cycle 
Management Strategic Consulting 2024–2025 Vendor Assessment 
Avasant RADAR Aerospace and Defense Digital Services 2024 RadarView™ 
IDC MarketScape IDC MarketScape: Worldwide Discrete Manufacturing PLM Strategic 
Consulting Services 2024 Vendor Assessment 
IDC MarketScape IDC MarketScape: Worldwide Discrete Manufacturing PLM Systems 
Integration 2024 Vendor Assessment 
IDC MarketScape IDC MarketScape: Worldwide Life Science R&D ITO Services 2024 
Vendor Assessment 
Avasant RADAR Life and Annuities Insurance Digital Services 2024–2025 
RadarView™ 
HFS Horizons HFS Horizons: Healthcare Payer Service Providers, 2024 
Everest PEAK Matrix Healthcare Provider Digital Services PEAK Matrix® Assessment 2024 
Avasant RADAR Higher Education Digital Services 2024 RadarView™  
Avasant RADAR Media and Entertainment Digital Services 2024 RadarView™ 
Avasant RADAR Governance, Risk, and Compliance Services 2024 RadarView™ 
HFS Horizons HFS Horizons: The Best Service Providers for Commercial Banks, 
2025 
Page 10 of 14 
 
Avasant RADAR Financial Services Digital Services 2024 RadarView™ 
IDC MarketScape IDC MarketScape: Worldwide Adobe Experience Cloud Professional 
Services 2024–2025 Vendor Assessment 
Avasant RADAR Digital Commerce Services 2024-2025 RadarView™ 
HFS Horizons HFS Horizons: IoT Service Providers, 2024 
Everest PEAK Matrix Digital Transformation Consulting Services PEAK Matrix® 
Assessment 2024 – Change Management 
Everest PEAK Matrix Digital Transformation Consulting Services PEAK Matrix® 
Assessment 2024 – Europe 
Avasant RADAR SAP S/4HANA Services 2024–2025 RadarView™ 
Gartner Magic 
Quadrant Magic Quadrant for Cloud ERP Services 
PAC Ranking 
Report SAP Services - Vendor Rankings - UK (Ranked 3rd) 
IDC MarketScape IDC MarketScape: European SAP Modernization Services 2024 
Vendor Assessment 
IDC MarketScape IDC MarketScape: Worldwide Cloud Security Services in the AI Era 
2024–2025 Vendor Assessment 
Everest PEAK Matrix Cybersecurity Services PEAK Matrix® Assessment 2024 – Europe 
Gartner Magic 
Quadrant Magic Quadrant for Service Integration and Management Services 
Avasant RADAR End-user Computing Services 2024–2025 RadarView™ 
Avasant RADAR Intelligent Automation Services 2024–2025 RadarView™ 
PAC Ranking 
Report Business Intelligence (BI) - Vendor Rankings - UK (Ranked 2nd) 
Avasant RADAR Hybrid Enterprise Cloud Services 2024-2025 RadarView™ 
IDC MarketScape IDC MarketScape: Asia/Pacific Managed Cloud Services 2024–2025 
Vendor Assessment 
Forrester Wave The Forrester Wave™ Infrastructure Outsourcing Services, Q4 2024 
PAC Ranking 
Report Big Data - Vendor Rankings - UK (Ranked 2nd) 
Avasant RADAR Intelligent ITOps Services 2024–2025 RadarView™ 
NelsonHall Ranking 
Report 
Finance and Accounting Transformation: From RPA to GenAI 
(Ranked 3rd) 
NelsonHall NEAT F&A Transformation 2024 
Everest PEAK Matrix Quality Engineering (QE) Services for AI Applications and Systems 
PEAK Matrix® Assessment 2024 
Everest PEAK Matrix Microsoft Azure Services PEAK Matrix® Assessment 2024 
HFS Horizons HFS Horizons: Azure Ecosystem™ Services Providers, 2024 
Everest PEAK Matrix Google Cloud Services PEAK Matrix® Assessment 2024 
IDC Ranking 
Report 
Worldwide Managed Cloud Services Market Shares, 2023: Top 10 
Worldwide Managed SPs (Ranked 2nd) 
PAC Ranking 
Report Cloud Ecosystem Services - Vendor Rankings - UK (Ranked 2nd) 
Page 11 of 14 
 
Everest PEAK Matrix AWS Services PEAK Matrix® Assessment 2024 
Everest PEAK Matrix Artificial Intelligence (AI) and Generative AI Services PEAK Matrix® 
Assessment 2024 
Avasant RADAR Generative AI Services 2024 RadarView™ 
HFS Horizons HFS Horizons: AADA Quadfecta Services for the Generative 
Enterprise™, 2024 
Everest PEAK Matrix Data and Analytics (D&A) Services PEAK Matrix® Assessment 2024 
IDC MarketScape IDC MarketScape: Worldwide Industry Cloud Professional Services 
2024 Vendor Assessment 
Everest PEAK Matrix Wealth Management Products PEAK Matrix® Assessment 2024 
Omdia Universe Omdia Universe: Payment Hubs, 2024–25 
Avasant RADAR Blockchain Services 2024–2025 RadarView™ 
*TCS rated #1 in the assessment. 
 
  
Page 12 of 14 
 
 
IFRS Financial Statements 
Consolidated Statements of Comprehensive Income 
For the three-month periods ended December 31, 2023, and December 31, 2024 
(In millions of ₹, except per share data) 
 
Three-month period ended 
December 31, 2023 
Three-month 
period  
ended December 
31, 2024 Ex Adj* Reported 
Revenue 605,830  605,830  639,730  
Cost of revenue 358,710  358,710  391,390  
Gross margin 247,120  247,120  248,340  
SG & A expenses 95,570  105,150  91,770  
Operating income 151,550  141,970  156,570  
Other income (expense), net 7,150  6,320  10,090  
Income before income taxes 158,700  148,290  166,660  
Income taxes 40,960  37,320  42,220  
Income after income taxes 117,740  110,970  124,440  
Non-controlling interests 390  390  640  
Net income 117,350  110,580  123,800  
Earnings per share in ₹ 32.14 30.29  34.21  
  *excludes settlement of legal claim 
  
Consolidated Statements of Financial Position 
As of March 31, 2024, and December 31, 2024 
(In millions of ₹) 
 As of March 31,  
2024 
As of December 31,  
2024 
Assets 
Property and equipment 112,240  112,670  
Right-of-use Assets 78,860  93,050  
Intangible assets and Goodwill 44,990  51,180  
Accounts Receivable 445,610  522,220  
Unbilled Revenues 153,000  145,470  
Invested Funds 469,630  623,490  
Other current assets 90,950  121,630  
Other non-current assets 90,440  94,630  
Total Assets 1,485,720  1,764,340  
Liabilities and Shareholders' Equity 
Shareholders' Funds 926,120  1,116,110  
Current liabilities 461,040  531,330  
Non-current liabilities 90,260  107,650  
Non-controlling interests 8,300  9,250  
Total Liabilities 1,485,720  1,764,340  
 
Page 13 of 14 
 
 
Ind AS Financial Statements 
Consolidated Statement of Profit and Loss 
For the Quarter ended December 31, 2023, and December 31, 2024 
(In ₹ crore, except per share data) 
  Quarter ended December 31, 
2023 
Quarter ended 
December 31, 
2024 Ex Adj* Reported 
Revenue 60,583  60,583  63,973  
Expenditure       
 a) Employee Costs 34,722  34,722  35,956  
 b) Cost of equipment and software licences 1,173  1,173  3,519  
 c) Other Operating expenses 8,300  8,300  7,464  
 d) Depreciation 1,233  1,233  1,377  
Total Expenditure  45,428  45,428  48,316  
Profit Before Taxes, Other Income & Exceptional Items 15,155  15,155  15,657  
Settlement of legal claim 0  958  0  
Profit Before Taxes & Other Income 15,155  14,197  15,657  
Other income (expense), net 715  632  1,009  
Profit Before Taxes 15,870  14,829  16,666  
Provision For Taxes 4,096  3,732  4,222  
Profit After Taxes & Before Non-controlling interests 11,774  11,097  12,444  
Non-controlling interests 39  39  64  
Net Profit 11,735  11,058  12,380  
 Earnings per share in ₹ 32.14  30.29  34.21  
 
Consolidated Balance Sheet 
As at March 31, 2024 and December 31, 2024 
(In crores of ₹) 
 As at March 31,  
2024  
As at December 31,  
2024 
ASSETS 
Property and equipment 10,940  10,907  
Right-of-use assets 7,886  9,305  
Intangible assets and Goodwill 2,342  2,959  
Accounts Receivable 44,561  52,222  
Unbilled Revenues 15,300  14,547  
Invested Funds 46,963  62,349  
Other current assets 9,095  12,168  
Other non-current assets 9,362  9,854  
Total Assets 146,449  174,311  
EQUITY AND LIABILITIES 
Shareholders' Funds 90,489  109,488  
Current liabilities 46,104  53,133  
Non-current liabilities 9,026  10,765  
Non-controlling interests 830  925  
Total Liabilities 146,449  174,311  
#### 
About Tata Consultancy Services 
Tata Consultancy Services is an IT services, consulting and business solutions organization that has 
been partnering with many of the world’s largest businesses in their transformation journeys for over 
56 years. Its consulting-led, cognitive powered, portfolio of business, technology and engineering 
services and solutions is delivered through its unique Location Independent Agile™ delivery model, 
recognized as a benchmark of excellence in software development. 
A part of the Tata group, India's largest multinational business group, TCS has over 601,000 of the 
world’s best-trained consultants in 55 countries. The company generated consolidated revenues of US 
$29 billion in the fiscal year ended March 31, 2024, and is listed on the BSE and the NSE in India. TCS' 
proactive stance on climate change and award-winning work with communities across the world have 
earned it a place in leading sustainability indices such as the MSCI Global Sustainability Index and the 
FTSE4Good Emerging Index. For more information, visit www.tcs.com  
TCS media contacts 
Asia Pacific  
Email: y.tham@tcs.com | Phone: +65 9270 4560 
Australia and New Zealand  
Email: kelly.ryan@tcs.com | Phone: +61 422 989 682 
Canada   
Email: tiffany.fisher@tcs.com | Phone: +1 416 456 7650  
Europe  
Email: roland.bagen@tcs.com | Phone: +46 70 3178024 
India  
Email: corporate.communications@tcs.com 
Email: saxena.kritika@tcs.com| Phone: +91 22 6778 9999 
Email: kimberly.solomon@tcs.com | Phone: +91 22 6778 9098 
Middle East & Africa  
Email: pragya.priyadarshini@tcs.com | Phone: +971 528656700  
Japan  
Email: douglas.foote@tcs.com | Phone: +81 80-2115-0989  
Latin America  
Email: alma.leal@tcs.com | Phone: +521 55 2095 6098  
UK 
Email: t.doherty@tcs.com | Phone: +44(0)7759396160 
USA   
Email: james.sciales@tcs.com | Phone: +1 917 981 7651  
### 
Page 14 of 14 """

In [46]:
from pydantic import BaseModel


class report(BaseModel):
    report_text: str

In [57]:
response = llm.with_structured_output(report).invoke(
    f"<system> You are an expert financial qualitative analyst. Your task is to generate a well-detailed deeply-researched finanical forecast for the company. </system> <text>{file_text}</text> Respond only with the content of the file in markdown format."
)

In [59]:
print(response)

None
